# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [6]:

%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

%%configure
{
    "--conf": "spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.sql.hive.convertMetastoreParquet=false",
    "--enable-glue-datacatalog" :"true",
    "--datalake-formats":"hudi"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
The following configurations have been updated: {'--conf': 'spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.sql.hive.convertMetastoreParquet=false', '--enable-glue-datacatalog': 'true', '--datalake-formats': 'hudi'}


####  Run this cell to set up and start your interactive session.


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 8156c067-f893-4988-b221-573daa2b48ac
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.sql.hive.convertMetastoreParquet=false
--datalake-formats hudi
Waiting for session 8156c067-f893-4988-b221-573daa2b48ac to get into ready status...
Session 8156c067-f893-4988-b221-573daa2b48ac has been created.



In [3]:
spark.sql("show databases;").show()

+---------+
|namespace|
+---------+
|  default|
|  hudidb1|
|  hudidb2|
|  hudidb3|
|  hudidb7|
+---------+


In [4]:
try:
    import os
    import sys


    import pyspark
    from pyspark import SparkConf, SparkContext
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, asc, desc
    from awsglue.utils import getResolvedOptions
    from awsglue.dynamicframe import DynamicFrame
    from awsglue.context import GlueContext

    #from faker import Faker

    print("All modules are loaded .....")

except Exception as e:
    print("Some modules are missing {} ".format(e))

All modules are loaded .....


In [5]:
database_name1 = "hudidb8"
table_name = "hudi_table"
base_s3_path = "s3a://test-ramneek-3"
final_base_path = "{base_s3_path}/{table_name}".format(
    base_s3_path=base_s3_path, table_name=table_name
)

In [6]:
class DataGenerator(object):

    @staticmethod
    def get_data():
        # Manually created data
        return [
            (1, "Alice Johnson", "IT", "CA", 120000, 30, 5000, 1677624870),
            (2, "Bob Smith", "HR", "NY", 90000, 40, 7000, 1677624871),
            (3, "Charlie Lee", "Sales", "TX", 110000, 35, 8000, 1677624872),
            (4, "David Brown", "Marketing", "FL", 95000, 29, 4000, 1677624873),
            (5, "Eve Davis", "IT", "IL", 105000, 32, 6000, 1677624874)
        ]

In [7]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
        .getOrCreate()
    return spark


spark = create_spark_session()
sc = spark.sparkContext
glueContext = GlueContext(sc)

In [8]:
hudi_options = {
    'hoodie.table.name': table_name,
    "hoodie.datasource.write.storage.type": "MERGE_ON_READ",
    'hoodie.datasource.write.recordkey.field': 'emp_id',
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field': 'ts',

    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': database_name1,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',

}

In [9]:
data = DataGenerator.get_data()

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
df = spark.createDataFrame(data=data, schema=columns)

In [10]:
df.show()

+------+-------------+----------+-----+------+---+-----+----------+
|emp_id|employee_name|department|state|salary|age|bonus|        ts|
+------+-------------+----------+-----+------+---+-----+----------+
|     1|Alice Johnson|        IT|   CA|120000| 30| 5000|1677624870|
|     2|    Bob Smith|        HR|   NY| 90000| 40| 7000|1677624871|
|     3|  Charlie Lee|     Sales|   TX|110000| 35| 8000|1677624872|
|     4|  David Brown| Marketing|   FL| 95000| 29| 4000|1677624873|
|     5|    Eve Davis|        IT|   IL|105000| 32| 6000|1677624874|
+------+-------------+----------+-----+------+---+-----+----------+


In [11]:
df.write.format("hudi").options(**hudi_options).mode("overwrite").save(final_base_path)

In [12]:
spark.sql("show databases;").show()


+---------+
|namespace|
+---------+
|  default|
|  hudidb1|
|  hudidb2|
|  hudidb3|
|  hudidb7|
|  hudidb8|
+---------+


In [13]:
spark.sql("use hudidb8;").show()

++
||
++
++


In [14]:
spark.sql("show tables;").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  hudidb8|hudi_table_ro|      false|
|  hudidb8|hudi_table_rt|      false|
+---------+-------------+-----------+


In [15]:
spark.sql("select * from hudi_table_rt;").show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+-------------+----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|employee_name|department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+-------------+----------+-----+------+---+-----+----------+
|  20240930073916918|20240930073916918...|                 1|                      |9f432bb5-43bd-474...|     1|Alice Johnson|        IT|   CA|120000| 30| 5000|1677624870|
|  20240930073916918|20240930073916918...|                 5|                      |9f432bb5-43bd-474...|     5|    Eve Davis|        IT|   IL|105000| 32| 6000|1677624874|
|  20240930073916918|20240930073916918...|                 3|                      |9f432bb5-43bd-474...|     3|  Charlie Lee|     Sales|   

In [16]:
s3_parquet_path = "s3://test-ramneek-3/hudi_table/9f432bb5-43bd-4748-88d8-829d4f0d73ba-0_0-30-157_20240930073916918.parquet"

# Read the Parquet file
df1 = spark.read.parquet(s3_parquet_path)

In [17]:
df1.show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+-------------+----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|employee_name|department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+-------------+----------+-----+------+---+-----+----------+
|  20240930073916918|20240930073916918...|                 1|                      |9f432bb5-43bd-474...|     1|Alice Johnson|        IT|   CA|120000| 30| 5000|1677624870|
|  20240930073916918|20240930073916918...|                 5|                      |9f432bb5-43bd-474...|     5|    Eve Davis|        IT|   IL|105000| 32| 6000|1677624874|
|  20240930073916918|20240930073916918...|                 3|                      |9f432bb5-43bd-474...|     3|  Charlie Lee|     Sales|   

In [18]:
impleDataUpd = [
    (6, "This is APPEND", "Sales", "RJ", 81000, 30, 23000, 827307999),
    (7, "This is APPEND", "Engineering", "RJ", 79000, 53, 15000, 1627694678),
]

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
usr_up_df = spark.createDataFrame(data=impleDataUpd, schema=columns)
usr_up_df.write.format("hudi").options(**hudi_options).mode("append").save(final_base_path)

In [19]:
###while we did append, one more parquet got created in s3, since it is only an append to the dataset, no delta log file is generated till this step


s3_parquet_path = "s3://test-ramneek-3/hudi_table/9f432bb5-43bd-4748-88d8-829d4f0d73ba-0_0-68-306_20240930074202318.parquet"

# Read the Parquet file
df2 = spark.read.parquet(s3_parquet_path)

df2.show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|  20240930073916918|20240930073916918...|                 1|                      |9f432bb5-43bd-474...|     1| Alice Johnson|         IT|   CA|120000| 30| 5000|1677624870|
|  20240930073916918|20240930073916918...|                 5|                      |9f432bb5-43bd-474...|     5|     Eve Davis|         IT|   IL|105000| 32| 6000|1677624874|
|  20240930073916918|20240930073916918...|                 3|                      |9f432bb5-43bd-474...|     3|   Charlie Lee|   

In [20]:

###update

impleDataUpd = [
    (3, "this is update on data lake", "Sales", "RJ", 81000, 30, 23000, 827307999),
]
columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
usr_up_df = spark.createDataFrame(data=impleDataUpd, schema=columns)
usr_up_df.write.format("hudi").options(**hudi_options).mode("append").save(final_base_path)

In [13]:
##after an update operation, no parquet file is generated, only one delta log file is generated. let;s view the real time table.

In [21]:
spark.sql("select * from hudi_table_rt;").show()   ###we can see, there is update at emp_id=3 record

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|       employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|  20240930073916918|20240930073916918...|                 1|                      |9f432bb5-43bd-474...|     1|       Alice Johnson|         IT|   CA|120000| 30| 5000|1677624870|
|  20240930073916918|20240930073916918...|                 5|                      |9f432bb5-43bd-474...|     5|           Eve Davis|         IT|   IL|105000| 32| 6000|1677624874|
|  20240930074340349|20240930074340349...|                 3|                      |9f432bb5-43bd-47

In [7]:
## hudi_table_ro will remain in its original form, to sync it we will need to do the compaction.

In [22]:
spark.sql("select * from hudi_table_ro;").show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|  20240930073916918|20240930073916918...|                 1|                      |9f432bb5-43bd-474...|     1| Alice Johnson|         IT|   CA|120000| 30| 5000|1677624870|
|  20240930073916918|20240930073916918...|                 5|                      |9f432bb5-43bd-474...|     5|     Eve Davis|         IT|   IL|105000| 32| 6000|1677624874|
|  20240930073916918|20240930073916918...|                 3|                      |9f432bb5-43bd-474...|     3|   Charlie Lee|   

In [24]:
"""
In Apache Hudi, the real-time (RT) table reflects the latest data, including all updates and deletes that have been performed. The read-optimized (RO) table, on the other hand, contains only the base data files, excluding any changes from the delta log files until compaction is triggered.

The compaction process merges the base data files with the log files (delta logs) to produce new base files that reflect the most up-to-date state of the data. The RO table will not reflect updates until this compaction is performed.

To synchronize your RO table with the RT table, you need to trigger a compaction. You can do this by setting up compaction in your Hudi options or triggering it manually.

Steps to Trigger Compaction Manually:
Update your Hudi write options to include compaction configuration, if it is not already set.
Run a compaction job to merge the log files and base files.

"""

'\nIn Apache Hudi, the real-time (RT) table reflects the latest data, including all updates and deletes that have been performed. The read-optimized (RO) table, on the other hand, contains only the base data files, excluding any changes from the delta log files until compaction is triggered.\n\nThe compaction process merges the base data files with the log files (delta logs) to produce new base files that reflect the most up-to-date state of the data. The RO table will not reflect updates until this compaction is performed.\n\nTo synchronize your RO table with the RT table, you need to trigger a compaction. You can do this by setting up compaction in your Hudi options or triggering it manually.\n\nSteps to Trigger Compaction Manually:\nUpdate your Hudi write options to include compaction configuration, if it is not already set.\nRun a compaction job to merge the log files and base files.\n\n'


In [27]:
# Load the Hudi table and list commits to find the commit time
meta_df = spark.read.format("hudi").load(final_base_path)
meta_df.createOrReplaceTempView("hudi_metadata")

# List all the commit times (the instant times)
commit_time_df = spark.sql("SELECT distinct(_hoodie_commit_time) as commit_time FROM hudi_metadata order by commit_time desc")
commit_time_df.show(truncate=False)


+-----------------+
|commit_time      |
+-----------------+
|20240930075453211|
|20240930074202318|
|20240930073916918|
+-----------------+


In [28]:
commit_time = "20240930073916918"  # Example commit time

# Perform a time travel query using the commit time
time_travel_df = spark.read.format("hudi") \
    .option("as.of.instant", commit_time) \
    .load(final_base_path)

# Display the results as of the commit time
time_travel_df.show(truncate=False)

+-------------------+---------------------+------------------+----------------------+-------------------------------------------------------------------------+------+-------------+----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                        |emp_id|employee_name|department|state|salary|age|bonus|ts        |
+-------------------+---------------------+------------------+----------------------+-------------------------------------------------------------------------+------+-------------+----------+-----+------+---+-----+----------+
|20240930073916918  |20240930073916918_0_0|1                 |                      |9f432bb5-43bd-4748-88d8-829d4f0d73ba-0_0-30-157_20240930073916918.parquet|1     |Alice Johnson|IT        |CA   |120000|30 |5000 |1677624870|
|20240930073916918  |20240930073916918_0_1|5                 |                      |9f432bb5-43

In [29]:
commit_time = "20240930074202318"  # Example commit time

# Perform a time travel query using the commit time
time_travel_df = spark.read.format("hudi") \
    .option("as.of.instant", commit_time) \
    .load(final_base_path)

# Display the results as of the commit time
time_travel_df.show(truncate=False)

+-------------------+---------------------+------------------+----------------------+-------------------------------------------------------------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                        |emp_id|employee_name |department |state|salary|age|bonus|ts        |
+-------------------+---------------------+------------------+----------------------+-------------------------------------------------------------------------+------+--------------+-----------+-----+------+---+-----+----------+
|20240930073916918  |20240930073916918_0_0|1                 |                      |9f432bb5-43bd-4748-88d8-829d4f0d73ba-0_0-68-306_20240930074202318.parquet|1     |Alice Johnson |IT         |CA   |120000|30 |5000 |1677624870|
|20240930073916918  |20240930073916918_0_1|5                 |                      |9f4

In [30]:
commit_time = "20240930075453211"  # Example commit time

# Perform a time travel query using the commit time
time_travel_df = spark.read.format("hudi") \
    .option("as.of.instant", commit_time) \
    .load(final_base_path)

# Display the results as of the commit time
time_travel_df.show(truncate=False)

+-------------------+---------------------+------------------+----------------------+--------------------------------------------------------------------------+------+---------------------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                         |emp_id|employee_name              |department |state|salary|age|bonus|ts        |
+-------------------+---------------------+------------------+----------------------+--------------------------------------------------------------------------+------+---------------------------+-----------+-----+------+---+-----+----------+
|20240930073916918  |20240930073916918_0_0|1                 |                      |9f432bb5-43bd-4748-88d8-829d4f0d73ba-0_0-163-635_20240930075214992.parquet|1     |Alice Johnson              |IT         |CA   |120000|30 |5000 |1677624870|
|20240930073916918  |20240930073

In [32]:
"""
spark.read.format("hudi").load(final_base_path) allows you to access both the data and the associated metadata in a unified way, leveraging Hudi's design to provide features like time travel and versioning efficiently. 

meta_df = spark.read.format("hudi").load(final_base_path):

This line reads the Hudi table from the S3 path (final_base_path) into a Spark DataFrame.
Hudi maintains metadata along with the data itself, including commit times (known as instant times) stored in the _hoodie_commit_time field.
The Hudi table can store multiple versions of the data through these commit times.
meta_df.createOrReplaceTempView("hudi_metadata"):

Here, a temporary SQL view called "hudi_metadata" is created from the DataFrame meta_df. This allows us to run SQL queries directly on the metadata of the Hudi table.
SQL queries are useful to manipulate and extract data in a tabular manner.
commit_time_df = spark.sql("SELECT distinct(_hoodie_commit_time) as commit_time FROM hudi_metadata order by commit_time desc"):

This SQL query fetches all distinct commit times (_hoodie_commit_time) from the Hudi table's metadata. It sorts them in descending order (order by commit_time desc), so the most recent commit appears first.
Commit times are important because Hudi stores different versions of the data at each commit. These commit times are critical for time travel queries.
commit_time_df.show(truncate=False):

This prints the distinct commit times (versions of the dataset) without truncating the results. This list shows all available points in time where changes (commits) were made to the dataset.
You’ll use one of these commit times to perform a time travel query."""

'\nmeta_df = spark.read.format("hudi").load(final_base_path):\n\nThis line reads the Hudi table from the S3 path (final_base_path) into a Spark DataFrame.\nHudi maintains metadata along with the data itself, including commit times (known as instant times) stored in the _hoodie_commit_time field.\nThe Hudi table can store multiple versions of the data through these commit times.\nmeta_df.createOrReplaceTempView("hudi_metadata"):\n\nHere, a temporary SQL view called "hudi_metadata" is created from the DataFrame meta_df. This allows us to run SQL queries directly on the metadata of the Hudi table.\nSQL queries are useful to manipulate and extract data in a tabular manner.\ncommit_time_df = spark.sql("SELECT distinct(_hoodie_commit_time) as commit_time FROM hudi_metadata order by commit_time desc"):\n\nThis SQL query fetches all distinct commit times (_hoodie_commit_time) from the Hudi table\'s metadata. It sorts them in descending order (order by commit_time desc), so the most recent comm

In [ ]:
"""
.option("as.of.instant", commit_time):

This is the key line for time travel in Hudi. By specifying as.of.instant, you are telling Hudi to load the table’s state as it existed at the exact commit_time provided.
The time travel query allows you to view the data as it was at that commit time.

"""

In [ ]:
###apache hudi cleaner